In [1]:
import sympy as sp
from numpy import array
import numpy as np
from sympy import init_printing
init_printing(use_latex='mathjax')
import scipy
from sklearn.metrics import mean_squared_error

sqrt2 = np.sqrt(2)
sqrt3 = np.sqrt(3)
upper_triangle_idx  =np.triu_indices(5)

In [2]:
# load data
hill_data_path = "../data_6x6/processed_data/hill48_bingo.csv"
hill_transposed_data_path = "../data_6x6/processed_data/hill48_bingo_transpose.csv"
data = np.loadtxt(hill_data_path)
transposed_data = np.loadtxt(hill_transposed_data_path)

In [3]:
# data, transposed_data = remove_hydrostatic_dependence(data_pre, transposed_data_pre)
eqps_0_data = np.isclose(data[:,6],0.0)
#print(eqps_0_data)
data = data[eqps_0_data,:]
eqps_0_trans_data = np.isclose(transposed_data[:,6],0.0)
transposed_data = transposed_data[eqps_0_trans_data, :]
data, transposed_data

(array([[ 0.79527073, -2.65570509,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  3.89279289,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  0.        , -3.44326339,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  4.50362436,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  0.        , -2.5779697 ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  0.        ,  2.5779697 ,  0.        ,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  0.        ,  0.        , -4.82293969,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  0.        ,  0.        ,  4.82293969,
          0.        ,  0.        ],
        [ 0.79527073,  0.        ,  0.        ,  0.        ,  0.        ,
         -6.82066673,  0.        ],
        [ 

In [4]:
def objective(params):
    #print(params)
    # fill a P matrix that has the params:
    candidate_P = np.array( [[ params[0], params[1], params[2], 0, 0, 0 ],
                             [ params[1], params[3], params[4], 0, 0, 0 ],
                             [ params[2], params[4], params[5], 0, 0, 0 ],
                             [ 0, 0, 0, params[6], 0, 0],
                             [ 0, 0, 0, 0, params[7], 0],
                             [0, 0, 0, 0, 0, params[8] ]
                             ] )
    #print(candidate_P)
    residual = np.zeros(len(data))
    for i, stress_vec in enumerate(data):
        sv = stress_vec[0:6]
        current_ys =  sv @ candidate_P @ sv 
        error = current_ys - 100

        residual[i] = np.abs(error)
    #print(scipy.linalg.norm(residual))
    return np.sum(residual)
    


In [5]:
initial_guess = np.array([ 8.5, -7, -1.5, 8.5, -1.5, 3, 2.0, 4.0, 2.0 ])
result = scipy.optimize.minimize(objective, initial_guess)
print(result)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 8.651535907233665e-06
        x: [ 1.100e+01 -7.000e+00 -4.000e+00  9.000e+00 -2.000e+00
             6.000e+00  1.400e+01  4.000e+00  2.000e+00]
      nit: 71
      jac: [-9.204e+01  8.806e+00 -7.684e+00 -6.246e+01  1.078e+01
             8.858e+01  3.024e+01 -8.500e+01  3.153e+02]
 hess_inv: [[ 1.529e-09 -5.965e-10 ...  4.733e-11 -9.932e-11]
            [-5.965e-10  9.454e-09 ... -2.348e-10  5.679e-10]
            ...
            [ 4.733e-11 -2.348e-10 ...  4.386e-11 -2.619e-11]
            [-9.932e-11  5.679e-10 ... -2.619e-11  8.369e-11]]
     nfev: 1422
     njev: 141


In [6]:
params = result.x
candidate_P = np.array( [[ params[0], params[1], params[2], 0, 0, 0 ],
                            [ params[1], params[3], params[4], 0, 0, 0 ],
                            [ params[2], params[4], params[5], 0, 0, 0 ],
                            [ 0, 0, 0, params[6], 0, 0],
                            [ 0, 0, 0, 0, params[7], 0],
                            [0, 0, 0, 0, 0, params[8] ]
                            ] )
print(candidate_P)
residual = np.zeros(len(data))
for i, stress_vec in enumerate(data):
    sv = stress_vec[0:6]
    current_ys =  sv @ candidate_P @ sv 
    error = current_ys - 100

    residual[i] = error
print(residual)

[[10.99999997 -6.99999996 -4.00000002  0.          0.          0.        ]
 [-6.99999996  8.99999999 -1.99999999  0.          0.          0.        ]
 [-4.00000002 -1.99999999  6.          0.          0.          0.        ]
 [ 0.          0.          0.         14.          0.          0.        ]
 [ 0.          0.          0.          0.          3.99999999  0.        ]
 [ 0.          0.          0.          0.          0.          2.        ]]
[-2.78678741e-07  2.47108716e-08  3.78275615e-08 -1.76659540e-07
 -3.71518212e-08 -3.71518212e-08 -2.45765577e-07 -2.45765577e-07
 -1.24430457e-07 -1.24430457e-07 -3.68970092e-07 -2.03112108e-07
 -7.07455854e-08 -9.63554214e-09 -2.40304274e-08 -2.40304274e-08
 -2.35479817e-07 -2.35479817e-07 -1.12494874e-07 -1.12494874e-07
 -1.80135771e-07 -4.43947030e-07 -1.53202492e-07 -1.25354461e-07
 -1.75138695e-08 -1.75138695e-08 -2.33217492e-07 -2.33217492e-07
 -1.07758012e-07 -1.07758012e-07 -2.83657059e-07 -2.83657059e-07
 -1.32854382e-07 -1.32854382e